In [5]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [3]:
#scrape a single game shots. We will allow users to enter a base url
base_url = 'https://understat.com/match/'
match = str(input('Please enter the match id: '))
url = base_url+match

Please enter the match id: 14898


In [26]:
#Use requests to get the webpage and BeautifulSoup to parse the page
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')
scripts = soup.find_all('script')

In [27]:
#get only the shotsData
strings = scripts[1].string

In [28]:
strings

"\n\tvar shotsData \t= JSON.parse('\\x7B\\x22h\\x22\\x3A\\x5B\\x7B\\x22id\\x22\\x3A\\x22392487\\x22,\\x22minute\\x22\\x3A\\x221\\x22,\\x22result\\x22\\x3A\\x22MissedShots\\x22,\\x22X\\x22\\x3A\\x220.8880000305175781\\x22,\\x22Y\\x22\\x3A\\x220.43099998474121093\\x22,\\x22xG\\x22\\x3A\\x220.110611192882061\\x22,\\x22player\\x22\\x3A\\x22Philippe\\x20Coutinho\\x22,\\x22h_a\\x22\\x3A\\x22h\\x22,\\x22player_id\\x22\\x3A\\x22488\\x22,\\x22situation\\x22\\x3A\\x22OpenPlay\\x22,\\x22season\\x22\\x3A\\x222020\\x22,\\x22shotType\\x22\\x3A\\x22RightFoot\\x22,\\x22match_id\\x22\\x3A\\x2214898\\x22,\\x22h_team\\x22\\x3A\\x22Barcelona\\x22,\\x22a_team\\x22\\x3A\\x22Levante\\x22,\\x22h_goals\\x22\\x3A\\x221\\x22,\\x22a_goals\\x22\\x3A\\x220\\x22,\\x22date\\x22\\x3A\\x222020\\x2D12\\x2D13\\x2020\\x3A00\\x3A00\\x22,\\x22player_assisted\\x22\\x3A\\x22Sergi\\x5Cu00f1o\\x20Dest\\x22,\\x22lastAction\\x22\\x3A\\x22Pass\\x22\\x7D,\\x7B\\x22id\\x22\\x3A\\x22392489\\x22,\\x22minute\\x22\\x3A\\x2212\\x22,\\x22

In [29]:
# strip unnecessary symbols and get only JSON data 
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')

#convert string to json format
data = json.loads(json_data)

In [30]:
data

{'h': [{'id': '392487',
   'minute': '1',
   'result': 'MissedShots',
   'X': '0.8880000305175781',
   'Y': '0.43099998474121093',
   'xG': '0.110611192882061',
   'player': 'Philippe Coutinho',
   'h_a': 'h',
   'player_id': '488',
   'situation': 'OpenPlay',
   'season': '2020',
   'shotType': 'RightFoot',
   'match_id': '14898',
   'h_team': 'Barcelona',
   'a_team': 'Levante',
   'h_goals': '1',
   'a_goals': '0',
   'date': '2020-12-13 20:00:00',
   'player_assisted': 'Sergiño Dest',
   'lastAction': 'Pass'},
  {'id': '392489',
   'minute': '12',
   'result': 'BlockedShot',
   'X': '0.855999984741211',
   'Y': '0.6759999847412109',
   'xG': '0.04891153424978256',
   'player': 'Lionel Messi',
   'h_a': 'h',
   'player_id': '2097',
   'situation': 'OpenPlay',
   'season': '2020',
   'shotType': 'LeftFoot',
   'match_id': '14898',
   'h_team': 'Barcelona',
   'a_team': 'Levante',
   'h_goals': '1',
   'a_goals': '0',
   'date': '2020-12-13 20:00:00',
   'player_assisted': 'Jordi Alba

In [31]:
x = []
y = []
xG = []
result = []
team = []
data_away = data['a']
data_home = data['h']

for index in range(len(data_home)):
    for key in data_home[index]:
        if key == 'X':
            x.append(data_home[index][key])
        if key == 'Y':
            y.append(data_home[index][key])
        if key == 'h_team':
            team.append(data_home[index][key])
        if key == 'xG':
            xG.append(data_home[index][key])
        if key == 'result':
            result.append(data_home[index][key])

for index in range(len(data_away)):
    for key in data_away[index]:
        if key == 'X':
            x.append(data_away[index][key])
        if key == 'Y':
            y.append(data_away[index][key])
        if key == 'a_team':
            team.append(data_away[index][key])
        if key == 'xG':
            xG.append(data_away[index][key])
        if key == 'result':
            result.append(data_away[index][key])

In [32]:
col_names = ['x','y','xG','result','team']
df = pd.DataFrame([x,y,xG,result,team],index=col_names)
df = df.T

In [33]:
df

,x,y,xG,result,team
0,0.8880000305175781,0.43099998474121093,0.110611192882061,MissedShots,Barcelona
1,0.855999984741211,0.6759999847412109,0.04891153424978256,BlockedShot,Barcelona
2,0.855999984741211,0.725999984741211,0.032116737216711044,SavedShot,Barcelona
3,0.8669999694824219,0.3220000076293945,0.04845098406076431,SavedShot,Barcelona
4,0.9109999847412109,0.43,0.29747647047042847,SavedShot,Barcelona
5,0.9690000152587891,0.5279999923706055,0.7536893486976624,MissedShots,Barcelona
6,0.89,0.51,0.08828572183847427,MissedShots,Barcelona
7,0.8230000305175781,0.3509999847412109,0.03806573152542114,MissedShots,Barcelona
8,0.7559999847412109,0.33,0.05780152603983879,BlockedShot,Barcelona
9,0.815,0.72,0.019371867179870605,SavedShot,Barcelona
